# Pour chaque ville / circuit, quel est le temps maximum pour être qualifié ?

## Temps maximum (en python pur)

Dans ce projet de traitement de données, il faut répondre à certaines questions en python sans pandas. On transforme donc le dataframe pandas en simples listes python.

In [30]:
# Fonction utilitaire pour fusionner deux listes de dictionnaires par une clé
def merge_dicts(left, right, key):
    right_lookup = {item[key]: item for item in right}
    return [{**item, **right_lookup[item[key]]} for item in left if item[key] in right_lookup]


# Fonction utilitaire pour lire et analyser les fichiers CSV manuellement
def read_csv(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        lines = f.readlines()
        header = lines[0].strip().split(",")
        return [dict(zip(header, line.strip().split(","))) for line in lines[1:]]


# Lire les données
qualif = read_csv("../data/qualifying.csv")
circuits = read_csv("../data/circuits.csv")
races = read_csv("../data/races.csv")

# Fusionner les courses et les circuits par circuitId
races_plus_circuits = merge_dicts(races, circuits, "circuitId")

# Préparer les données intéressantes avec les temps de qualification et les références des circuits
interesting_datas = []
for row in qualif:
    race_id = row["raceId"]
    # Find the corresponding race data (merged with circuits)
    race_data = next((race for race in races_plus_circuits if race["raceId"] == race_id), None)
    if race_data:
        interesting_datas.append({
            "circuitId": race_data["circuitId"],
            "circuitRef": race_data["circuitRef"],
            "q1": row["q1"].strip('"').strip() if row["q1"] != "\\N" else None,
            "q2": row["q2"].strip('"').strip() if row["q2"] != "\\N" else None,
            "q3": row["q3"].strip('"').strip() if row["q3"] != "\\N" else None
        })


# Fonction utilitaire pour convertir le temps du format "mm:ss.sss" en secondes totales (float)
def time_to_seconds(time_str):
    if not time_str or time_str in ("\\N", ""):
        return None
    try:
        # Split into minutes, seconds and milliseconds
        minutes, seconds = time_str.split(":")
        seconds, milliseconds = seconds.split(".")
        # Convert to total seconds
        total_seconds = int(minutes) * 60 + int(seconds) + int(milliseconds) / 1000
        return total_seconds
    except ValueError:
        return None  # If the time is malformed, return None

# Remplacer '\N' par None et convertir en secondes
for data in interesting_datas:
    data["q1"] = time_to_seconds(data["q1"]) if data["q1"] else None
    data["q2"] = time_to_seconds(data["q2"]) if data["q2"] else None
    data["q3"] = time_to_seconds(data["q3"]) if data["q3"] else None

# Préparer les colonnes pour les temps
q1_column = sorted([data["q1"] for data in interesting_datas if data["q1"] is not None])
q1_column.pop() # REMOVED the weird value created by merge (TODO)
q2_column = [data["q2"] for data in interesting_datas if data["q2"] is not None]
q3_column = [data["q3"] for data in interesting_datas if data["q3"] is not None]



# Trouver les temps maximum de qualification
max_q1 = max(q1_column) if q1_column else None
max_q2 = max(q2_column) if q2_column else None
max_q3 = max(q3_column) if q3_column else None

# Output the results
print(f"Max Q1: {max_q1}, Max Q2: {max_q2}, Max Q3: {max_q3}")

# Regrouper par circuitId pour trouver les temps maximum de qualification par circuit
circuit_times = {}
for data in interesting_datas:
    circuit_id = data["circuitId"]
    if circuit_id.lower() not in map(str.lower, circuit_times.keys()):
        circuit_times[circuit_id] = {"q1": [], "q2": [], "q3": []}

    # Append times to the appropriate circuit
    if data["q1"] is not None:
        circuit_times[circuit_id]["q1"].append(data["q1"])
    if data["q2"] is not None:
        circuit_times[circuit_id]["q2"].append(data["q2"])
    if data["q3"] is not None:
        circuit_times[circuit_id]["q3"].append(data["q3"])

# Calculer le temps maximum par circuit
for circuit_id, times in circuit_times.items():
    max_q1_circuit = max(times["q1"]) if times["q1"] else None
    max_q2_circuit = max(times["q2"]) if times["q2"] else None
    max_q3_circuit = max(times["q3"]) if times["q3"] else None
    print(f"Circuit {circuit_id}: Max Q1: {max_q1_circuit}, Max Q2: {max_q2_circuit}, Max Q3: {max_q3_circuit}")

Max Q1: 153.885, Max Q2: 132.47, Max Q3: 129.776
Circuit 1: Max Q1: 117.931, Max Q2: 107.293, Max Q3: 108.147
Circuit 2: Max Q1: 124.407, Max Q2: 122.885, Max Q3: 124.053
Circuit 3: Max Q1: 124.904, Max Q2: 116.619, Max Q3: 116.309
Circuit 4: Max Q1: 92.038, Max Q2: 88.28, Max Q3: 87.402
Circuit 5: Max Q1: 141.611, Max Q2: 118.556, Max Q3: 117.226
Circuit 6: Max Q1: 91.542, Max Q2: 86.632, Max Q3: 78.327
Circuit 7: Max Q1: 96.575, Max Q2: 93.127, Max Q3: 91.349
Circuit 8: Max Q1: 84.865, Max Q2: 77.105, Max Q3: 78.663
Circuit 9: Max Q1: 114.143, Max Q2: 117.895, Max Q3: 123.095
Circuit 10: Max Q1: 114.625, Max Q2: 101.551, Max Q3: 105.811
Circuit 11: Max Q1: 110.189, Max Q2: 97.075, Max Q3: 100.593
Circuit 12: Max Q1: 104.363, Max Q2: 100.358, Max Q3: 100.75
Circuit 13: Max Q1: 153.885, Max Q2: 132.47, Max Q3: 129.776
Circuit 14: Max Q1: 103.355, Max Q2: 99.265, Max Q3: 99.906
Circuit 15: Max Q1: 117.532, Max Q2: 115.518, Max Q3: 111.983
Circuit 16: Max Q1: 89.668, Max Q2: 86.948, Max 